# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
"
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk 
".


# Feladatok

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint. Melyik államban van a legtöbb, legkevesebb egyetem?


In [ ]:
with open('/content/sample_data/egyetemek.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

universities = {}

for line in data:
    line = line.strip()
    if line:
        try:
            state, unis = line.split("[edit]")
            state = state.strip()
            unis = unis.strip().strip("[]")
            uni_list = unis.split(",")
            universities[state] = [uni.strip() for uni in uni_list]
        except ValueError:
            print(f"Could not process line: {line}")

for line in data:
    line = line.strip()
    if line:
        state, unis = line.split("[edit]") if "[edit]" in line else (line, "")
        state = state.strip()
        unis = unis.strip().strip("[]")
        uni_list = unis.split(",")
        universities[state] = [uni.strip() for uni in uni_list]

counts = {state: len(unis) for state, unis in universities.items()}

most_unis_state = max(counts, key=counts.get)
least_unis_state = min(counts, key=counts.get)

print("**********************************************************")
print(f"Most Uni: {most_unis_state} ({counts[most_unis_state]} db)")
print(f"Least Uni: {least_unis_state} ({counts[least_unis_state]} db)")
print("**********************************************************")

# print(universities)

2. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [ ]:
df_codes = pd.read_csv('/content/content/sample_data/shortNames.csv',names = ['State', 'Code'])
df = df.merge(df_codes, on = 'State', how='left')

In [ ]:
d = df[['State','Code']].copy()
d['szam'] = 1
uniabs = d.groupby(['State']).sum()
uniabs = uniabs.merge(df_codes, on = 'State', how='left')
uniabs

3. Az adatbázist integráljuk a [List of states and territories of the United States by population](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population) linken szereplő népszámlálási adatokkal és számoljuk ki államonként hány főre jut egy egyetemi város.

In [ ]:
df_pop = pd.read_csv('/content/sample_data/popular.csv',names = ['State', 'Popular'])

In [ ]:
d = df[['State','Code']].copy()
d['szam'] = 1
by_state = d.groupby(['State'])
by_state = by_state.sum()
new_pop = by_state.merge(df_pop, on = 'State', how='left')
new_pop = new_pop.merge(df_codes, on = 'State', how='left')
new_pop['fo/egyet'] = new_pop['Popular']/new_pop['szam']
new_pop.head()


4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatok integrálásával, számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

In [ ]:
df_area = pd.read_csv('/content/sample_data/area.csv',names = ['State', 'Area'])

In [4]:
d = df[['State']].copy()
d['szam'] = 1
by_state = d.groupby(['State'])
by_state = by_state.sum()
area = by_state.merge(df_area, on = 'State', how='left')
area = area.merge(df_codes, on = 'State', how='left')
area['ter/egyet'] = area['Area']//area['szam']
area.head()

NameError: ignored

5. A [példa](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/), a kivitelt, exportmennyiséget ábrázolja az Amerikai Egyesült Államok térképen, államonként lebontva ezt. Készítsünk hasonló ábrákat az egyetemek abszolút, lakoság és terület szerinti eloszlásáról is. Az ábrákat exportáljuk kép formájában.
![](https://i.ibb.co/s1zdnLY/mapplot.png)

Ábra 1. [Térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével. [Forrás](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/).

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data = go.Choropleth(
    locations = area['Code'],
    z = area['ter/egyet'],
    locationmode = 'USA-states',
    colorscale = 'Reds',
))
fig.update_layout(
    title_text = '2011 US Area/University by State',
    geo_scope = 'usa',
)
fig.show()

In [ ]:
fig = go.Figure(data = go.Choropleth(
    locations = new_pop['Code'],
    z = new_pop['fo/egyet'],
    locationmode = 'USA-states',
    colorscale = 'Reds',
))
fig.update_layout(
    title_text = '2011 US Popular/University by State',
    geo_scope = 'usa',
)
fig.show()

In [ ]:
fig = go.Figure(data = go.Choropleth(
    locations = uniabs['Code'],
    z = uniabs['szam'],
    locationmode = 'USA-states',
    colorscale = 'Reds',
))
fig.update_layout(
    title_text = '2011 US Absalut University by State',
    geo_scope = 'usa',
)
fig.show()

6. Számoljuk ki, hány egyetem van városonként, majd összesítve államonként. Ehhez az `egyetemek.txt` fájlban miután megkapunk egy várost, az utána következő kerek zárójelek között megszámoljuk, hány vesszővel elválasztott karakterlánc található. 
Pl. a következő sorban:	`Claremont (Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University)[5]` -> 6 vessző van a kerek zárójelek között, tehát 7 egyetem van a városban.    
Melyik államban és melyik városban van a legtöbb egyetem?

In [ ]:
def clean_town(item):
  return item[:item.find('(') - 1]
def clean_state(item):
  return item[:item.find('[')]
def what_univers(item):
  return len(item[item.find('(') - 1:].split(','))

df1['NumberUniv'] = df1.Town.apply(what_univers)
df1['Town'] = df1.Town.apply(clean_town)
df1['State'] = df1.State.apply(clean_state)
df1

In [ ]:
by_state = df1.groupby(['State'])
maxi = by_state.NumberUniv.sum().max()
mini = by_state.NumberUniv.sum().min()
print(maxi,mini)

by_town = df1.groupby(['Town'])
maxi = by_town.NumberUniv.sum().max()
mini = by_town.NumberUniv.sum().min()
print(maxi,mini)